### Preparing HDFS
Using magic

Create input folder on HDFS if not exists

Copy from data from local

In [12]:
! hadoop fs -mkdir -p  /tmp/input
! hadoop fs -put   -p  ./../data-clean/*.csv             /tmp/input         

put: `/tmp/input/cleanMelbourneData.csv': File exists




In [13]:
// Load Clean Dataset into a DataFrame from HDFS after wrangling is completed
var df_clean = spark
    .read
    .format("csv")
    .option("header", "true")
    .load("hdfs://localhost:9000/tmp/input/*.csv")

df_clean: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [14]:
df_clean = df_clean.withColumn("Price",col("Price").cast("Double"))
    .withColumn("Rooms",col("Rooms").cast("Int"))
    .withColumn("DistanceFromCBD",col("DistanceFromCBD").cast("Double"))
    .withColumn("MethodOfSale",col("MethodOfSale").cast("Int"))
    .withColumn("PropertyType",col("PropertyType").cast("Int"))
    .withColumn("Bathroom",col("Bathroom").cast("Int"))
    .withColumn("Car",col("Car").cast("Int"))
    .withColumn("Landsize",col("Landsize").cast("Double"))
    .withColumn("Latitude",col("Latitude").cast("Double"))
    .withColumn("Longtitude",col("Longtitude").cast("Double"))



df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [15]:
df_clean.printSchema()

root
 |-- Price: double (nullable = true)
 |-- MethodOfSale: integer (nullable = true)
 |-- PropertyType: integer (nullable = true)
 |-- DistanceFromCBD: double (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Bathroom: integer (nullable = true)
 |-- Car: integer (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- StreetName: string (nullable = true)



### Change attributes into vectors
#### Transform Sale Date into a numeric value

In [16]:
df_clean = df_clean.withColumn("Date",unix_timestamp($"Date", "dd/mm/yyyy"))

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


#### Set FeatureHasher for Suburb, StreetName

In [17]:
import org.apache.spark.ml.feature.{FeatureHasher,OneHotEncoder,StandardScaler,VectorAssembler}

val hasher = new FeatureHasher()
 .setInputCols("StreetName","Suburb")
 .setOutputCol("str_name_suburb_vec")

import org.apache.spark.ml.feature.{FeatureHasher, OneHotEncoder, StandardScaler, VectorAssembler}
hasher: org.apache.spark.ml.feature.FeatureHasher = featureHasher_5735c63a7d27


#### Set OneHotEncoders for PropertyType, MethodOfSale

In [18]:
val ms_encoder = new OneHotEncoder()
      .setInputCol("MethodOfSale")
      .setOutputCol("m_sale_vec")

val pt_encoder = new OneHotEncoder()
      .setInputCol("PropertyType")
      .setOutputCol("pt_vec")

ms_encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_a1f3c99429b9
pt_encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_5843e222abf3


#### Assemble the columns and column vectors into a single column - "features"

In [19]:
val columns = Array("Price", "DistanceFromCBD", "Rooms", "Bathroom", "Car", "Landsize", "Latitude", "Longtitude", "Date", 
                    "str_name_suburb_vec", "m_sale_vec", "pt_vec")

val assembler = new VectorAssembler()
            .setInputCols(columns)
            .setOutputCol("features")

columns: Array[String] = Array(Price, DistanceFromCBD, Rooms, Bathroom, Car, Landsize, Latitude, Longtitude, Date, str_name_suburb_vec, m_sale_vec, pt_vec)
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_8a045747b379


#### Set StandardScaler

In [20]:
var scaler = new StandardScaler()
      .setInputCol("features")
      .setOutputCol("ScaledFeatures")
      .setWithStd(true).setWithMean(true)

scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_0e120a6494eb


### Split Data into a Training and a Testing Set

In [21]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


def train_test_split(data: DataFrame) = {
    
     val Array(train, test) = data.randomSplit(Array(0.8, 0.2), seed = 30)
    
     (train, test)
}

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
train_test_split: (data: org.apache.spark.sql.DataFrame)(org.apache.spark.sql.Dataset[org.apache.spark.sql.Row], org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])


In [22]:

val (train, test) = train_test_split(df_clean)

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double, MethodOfSale: int ... 11 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double, MethodOfSale: int ... 11 more fields]


### 1. Apply Linear Regression


In [23]:
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.Pipeline



val lr = new LinearRegression()
    .setLabelCol("Price")
    .setFeaturesCol("features")
    .setPredictionCol("Predicted Price")
    .setMaxIter(50)

import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.Pipeline
lr: org.apache.spark.ml.regression.LinearRegression = linReg_9e079f937458


#### Set Stages to transform all columns into feature

In [24]:
// add linear regression to stages
val lrStages = Array(
            hasher,
            ms_encoder, 
            pt_encoder,
            assembler,
            //scaler,
            lr
)

lrStages: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(featureHasher_5735c63a7d27, oneHot_a1f3c99429b9, oneHot_5843e222abf3, vecAssembler_8a045747b379, linReg_9e079f937458)


#### Construct the pipeline

In [25]:
val startTimeMillis = System.currentTimeMillis()

val lrPipe = new Pipeline().setStages(lrStages)

//We fit our DataFrame into the pipeline to generate a model
val lrModel = lrPipe.fit(train)

//Make predictions using the model and the test data
val predictions = lrModel.transform(test)

val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
print("pipeline was executed "+durationSeconds)
20

2020-05-30 03:46:52,237 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-05-30 03:46:52,237 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
pipeline was executed 115

startTimeMillis: Long = 1590810400873
lrPipe: org.apache.spark.ml.Pipeline = pipeline_342801117ebd
lrModel: org.apache.spark.ml.PipelineModel = pipeline_342801117ebd
predictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 16 more fields]
endTimeMillis: Long = 1590810516511
durationSeconds: Long = 115
res3: Int = 20


#### Prediction

In [26]:
predictions.columns

res4: Array[String] = Array(Price, MethodOfSale, PropertyType, DistanceFromCBD, Rooms, Bathroom, Car, Landsize, Latitude, Longtitude, Suburb, Date, StreetName, str_name_suburb_vec, m_sale_vec, pt_vec, features, Predicted Price)


In [27]:
predictions.withColumn("Predicted Price", round($"Predicted Price", 0)).select("Price","Predicted Price").show()

+--------+---------------+
|   Price|Predicted Price|
+--------+---------------+
|170000.0|       170064.0|
|280000.0|       280013.0|
|280500.0|       280579.0|
|283000.0|       283663.0|
|290000.0|       290036.0|
|300000.0|       300055.0|
|300000.0|       299881.0|
|305000.0|       305248.0|
|310000.0|       310135.0|
|316000.0|       315935.0|
|320000.0|       320027.0|
|320000.0|       320035.0|
|320000.0|       320560.0|
|320000.0|       319303.0|
|325000.0|       325090.0|
|333000.0|       332989.0|
|340000.0|       342929.0|
|345000.0|       345065.0|
|348000.0|       348240.0|
|350000.0|       350108.0|
+--------+---------------+
only showing top 20 rows



#### Evaluation

In [28]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

def evaluate ( predictions: DataFrame, metric: String) = {
    val eval =  new RegressionEvaluator()
       .setLabelCol("Price")
       .setPredictionCol("Predicted Price")
       .setMetricName(metric)
println("Root Mean Squared Error "+  metric.toUpperCase()+" on test data = " + eval.evaluate(predictions))
    
}

import org.apache.spark.ml.evaluation.RegressionEvaluator
evaluate: (predictions: org.apache.spark.sql.DataFrame, metric: String)Unit


#### Regression metrics

**Mean squared error (MSE)** -- the average of squared differences between the predicted outcome and the true outcome.

**R2 coefficient** -- the proportion of variance in the outcome that our model is capable of predicting based on its features.


In [29]:
evaluate(predictions,"rmse")

Root Mean Squared Error RMSE on test data = 975.1264181243374


In [30]:
evaluate(predictions,"r2")

Root Mean Squared Error R2 on test data = 0.9999977135307482


#### Testing/Evaluation/ Parameter Tuning

Cross-validation

<span style="color:red">
TO DO: does not finish run in reasonable time
</span>

In [ ]:
import org.apache.spark.ml.Predictor
import org.apache.spark.ml.PredictionModel
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression()
    .setLabelCol("Price")
    .setFeaturesCol("features")
    .setPredictionCol("Prediction")


val lrParamMap = new ParamGridBuilder()
    .addGrid(lr.regParam, Array(10, 1, 0.1, 0.01, 0.001))
    .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
    .addGrid(lr.maxIter, Array(10000, 250000))
    .build()

val columns = Array("Price", "DistanceFromCBD", "Rooms", "Bathroom", "Car", "Landsize", "Latitude", "Longtitude", "Date", 
                    "str_name_suburb_vec", "m_sale_vec", "pt_vec")

val assembler = new VectorAssembler()
            .setInputCols(columns)
            .setOutputCol("features")


val lrStages = Array(
            hasher,
            ms_encoder, 
            pt_encoder,
            assembler,
            //scaler,
            lr
)

val lrPipeline = new Pipeline()
    .setStages(lrStages)

val cvLR = new CrossValidator()
    .setEstimator(lrPipeline)
    .setEvaluator(new RegressionEvaluator()
    .setLabelCol("Price")
    .setPredictionCol("Prediction")
    .setMetricName("rmse"))
    .setEstimatorParamMaps(lrParamMap)
    .setNumFolds(5)
    .setParallelism(2)

val startTimeMillis = System.currentTimeMillis()

val cvLRModel = cvLR.fit(train)
val lrPredictionsAndPrice = cvLRModel
    .transform(test)


val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
print("pipeline was executed "+durationSeconds)

#### Parameter Tuning

In [ ]:
lrPredictionsAndPrice.show()
val bestLRModel = cvLRModel.bestModel
    
println(bestLRModel.extractParamMap)

In [ ]:
lrPredictionsAndPrice.withColumn("Predicted Price", round($"Predicted Price", 0)).select("Price","Predicted Price").show()


### 2. Apply KNN

#### Training


Pipeline Estimator

#### Prediction

#### Testing/Evaluation

Pipeline Model Transformer

### 3. Apply Random Forest Regression

**Build Random Forest model**
Specify maxDepth, maxBins, impurity, auto and seed parameters.

**maxDepth** -- Maximum depth of a tree. Increasing the depth makes the model more powerful, but deep trees take longer to train.

**maxBins** -- Maximum number of bins used for discretizing continuous features and for choosing how to split on features at each node.

**impurity** -- Criterion used for information gain calculation

**auto** -- Automatically select the number of features to consider for splits at each tree node

**seed** -- Use a random seed number , allowing to repeat the results

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.ml.Pipeline

val seed = 5043

val rf = new RandomForestRegressor()
  .setMaxBins(4)
  .setMaxDepth(2)
  .setNumTrees(10)
  .setFeatureSubsetStrategy("auto")
  .setSeed(seed)
  .setLabelCol("Price")
  .setFeaturesCol("features")
  .setPredictionCol("Predicted Price")

val rfStages = Array(
            hasher,
            ms_encoder, 
            pt_encoder,
            assembler,
            //scaler,
            rf
)

val rfPipe = new Pipeline().setStages(rfStages) 

val startTimeMillis = System.currentTimeMillis()
val rfModel = rfPipe.fit(train)

//Make predictions using the model and the test data
val rfPredictions = rfModel.transform(test)


val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
print("pipeline was executed "+durationSeconds)

2020-05-30 03:52:06,525 WARN  [Executor task launch worker for task 76] memory.MemoryStore (Logging.scala:logWarning(66)) - Not enough space to cache rdd_186_0 in memory! (computed 272.9 MB so far)
2020-05-30 03:52:06,531 WARN  [Executor task launch worker for task 76] storage.BlockManager (Logging.scala:logWarning(66)) - Persisting block rdd_186_0 to disk instead.
2020-05-30 03:52:45,388 WARN  [Executor task launch worker for task 76] memory.MemoryStore (Logging.scala:logWarning(66)) - Not enough space to cache rdd_186_0 in memory! (computed 272.9 MB so far)
2020-05-30 03:54:18,239 WARN  [Executor task launch worker for task 78] memory.MemoryStore (Logging.scala:logWarning(66)) - Not enough space to cache rdd_186_0 in memory! (computed 272.9 MB so far)
2020-05-30 03:54:19,883 ERROR [Executor task launch worker for task 78] executor.Executor (Logging.scala:logError(91)) - Exception in task 0.0 in stage 78.0 (TID 78)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.tr

	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.tree.impl.DTStatsAggregator.<init>(DTStatsAggregator.scala:77)
	at org.apache.spark.ml.tree.impl.RandomForest$$anonfun$13$$anonfun$14.apply(RandomForest.scala:545)
	at org.apache.spark.ml.tree.impl.RandomForest$$anonfun$13$$anonfun$14.apply(RandomForest.scala:541)
	at scala.Array$.tabulate(Array.scala:331)
	at org.apache.spark.ml.tree.impl.RandomForest$$anonfun$13.apply(RandomForest.scala:541)
	at org.apache.spark.ml.tree.impl.RandomForest$$anonfun$13.apply(RandomForest.scala:538)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:823)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:823)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:346)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:310)
	at org.a

In [ ]:
rfPredictions.columns

In [ ]:
rfPredictions.withColumn("Predicted Price", round($"Predicted Price", 0)).select("Price","Predicted Price").show()

In [ ]:
evaluate(rfPredictions,"rmse")

In [ ]:
evaluate(rfPredictions,"r2")

#### Testing/Evaluation/ Parameter Tuning

Cross-validation
<span style="color:red">
TO DO: 
* finish implementation for Cross-validation 
* check if finish run in reasonable time
</span>

In [ ]:
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.sql.{ DataFrame, Row, SQLContext }
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
import org.apache.spark.ml.Predictor
import org.apache.spark.ml.PredictionModel
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.Pipeline


// Models hypoparameters
val numTrees = Seq(5,10,15)
val maxBins = Seq(2,5,10)
val maxDepth = Seq(2,3,5)
val impurity = Seq("gini","entropy","variance",)
val featureSubsetStrategy = Seq("sqrt")

val rf = new RandomForestRegressor()
  .setLabelCol("Price")
  .setFeaturesCol("features")
  .setPredictionCol("Predicted Price")


val lrParamMap = new ParamGridBuilder()
    .addGrid(lr.regParam, Array(10, 1, 0.1, 0.01, 0.001))
    .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
    .addGrid(lr.maxIter, Array(10000, 250000))
    .build()

val rfParamMap = new ParamGridBuilder()
                      .addGrid(rf.numTrees, numTrees)
                      .addGrid(rf.maxDepth, maxDepth)
                      .addGrid(rf.impurity, impurity)
                      .addGrid(rf.maxBins, maxBins)
                      .addGrid(rf.featureSubsetStrategy, featureSubsetStrategy)
                      .build()


val columns = Array("Price", "DistanceFromCBD", "Rooms", "Bathroom", "Car", "Landsize", "Latitude", "Longtitude", "Date", 
                    "str_name_suburb_vec", "m_sale_vec", "pt_vec")

val assembler = new VectorAssembler()
            .setInputCols(columns)
            .setOutputCol("features")


val rfStages = Array(
            hasher,
            ms_encoder, 
            pt_encoder,
            assembler,
            //scaler,
            rf
)

val rfPipeline = new Pipeline()
    .setStages(rfStages)

val cvRF = new CrossValidator()
    .setEstimator(rfPipeline)
    .setEvaluator(new RegressionEvaluator()
    .setLabelCol("Price")
    .setPredictionCol("Prediction")
    .setMetricName("rmse"))
    .setEstimatorParamMaps(rfParamMap)
    .setNumFolds(5)
    .setParallelism(2)

val startTimeMillis = System.currentTimeMillis()
val cvRFModel = cvRF.fit(train)
val rFPredictionsAndPrice = cvRFModel
    .transform(test)


val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
print("pipeline was executed "+durationSeconds)


#### Prediction

In [ ]:
rfPredictions.withColumn("Predicted Price", round($"Predicted Price", 0)).select("Price","Predicted Price").show()

// this will add new columns rawPrediction, probability and prediction
val predictionDf = randomForestModel.transform(testData)
predictionDf.show(10)

#### Tuning

In [ ]:
rfPredictions.withColumn("Predicted Price", round($"Predicted Price", 0)).select("Price","Predicted Price").show()

// this will add new columns rawPrediction, probability and prediction
val predictionDf = randomForestModel.transform(testData)
predictionDf.show(10)

#### Bias vs Variance Graph of Error (validation error and training error) versus training set size. 


<span style="color:red">
TO DO: 
produce graph -- validation error and training errorshould converge
</span>


### References

Apache Spark (n.d.). _Spark ML Programming Guide._ Retrieved from https://spark.apache.org/docs/1.2.2/ml-guide.html

Hydrospheredata (n.d.). _org.apache.spark.ml.feature.StandardScaler Scala Examples._ Retrieved from https://towardsdatascience.com/from-scikit-learn-to-spark-ml-f2886fb46852

Johnson S (2019). _From sckit-learn to Spark ML._ Retrieved from https://www.programcreek.com/scala/org.apache.spark.ml.feature.StandardScaler
Masri A. (2019). _FeatureTransformation._ Retrieved from https://towardsdatascience.com/apache-spark-mllib-tutorial-7aba8a1dce6e
